In [1]:
import os
import pandas as pd
import numpy as np
import pandas as pd
from sklearn import preprocessing

import matplotlib.pyplot as plt
plt.rc("font", size=14)
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

AttributeError: module 'numpy' has no attribute 'MachAr'

In [2]:
import cv2

ImportError: dlopen(/Users/johndufek/opt/anaconda3/lib/python3.8/site-packages/cv2/python-3.8/cv2.cpython-38-darwin.so, 2): Library not loaded: @rpath/libgstreamer-1.0.0.dylib
  Referenced from: /Users/johndufek/opt/anaconda3/lib/libopencv_gapi.4.6.0.dylib
  Reason: Incompatible library version: libopencv_gapi.406.dylib requires version 1401.0.0 or later, but libgstreamer-1.0.0.dylib provides version 0.0.0

In [20]:
train_img = []
train_labels = []

test_img = []
test_labels = []

path_train = ('/Users/johndufek/Downloads/archive (2)/Training/')
path_test = ('/Users/johndufek/Downloads/archive (2)/Testing/')
img_size= 300

for i in os.listdir(path_train):
    for j in os.listdir(path_train+i):
        train_img.append (cv2.resize(cv2.imread(path_train+i+'/'+j), (img_size,img_size))) 
        train_labels.append(i)
        
for i in os.listdir(path_test):
    for j in os.listdir(path_test+i):
        test_img.append (cv2.resize(cv2.imread(path_test+i+'/'+j), (img_size,img_size))) 
        test_labels.append(i)
        
train_img = (np.array(train_img))
test_img = (np.array(test_img))


train_labels_encoded = [0 if category == 'no_tumor' else(1 if category == 'glioma_tumor' else(2 if category=='meningioma_tumor' else 3)) for category in list(train_labels)]
test_labels_encoded = [0 if category == 'no_tumor' else(1 if category == 'glioma_tumor' else(2 if category=='meningioma_tumor' else 3)) for category in list(test_labels)]



NameError: name 'cv2' is not defined